In [ ]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [17]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_03 import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

**DATABUNCH/ LEARNER**

In [22]:
#export
class Dataset():
  def __init__(self, x, y): self.x,self.y = x,y
  def __len__(self): return len(self.x)
  def __getitem__(self, i): return self.x[i], self.y[i]

def get_dls(train_ds, valid_ds, bs, **kwargs):
  return (DataLoader(train_ds, batch_size=bs,shuffle= True, **kwargs),
          DataLoader(valid_ds, batch_size=bs*2, **kwargs))

In [19]:
x_train, y_train, x_valid, y_valid = get_data()
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh,bs = 50,64
c= y_train.max().item()+1
loss_func = F.cross_entropy

In [20]:
#export
class DataBunch():
  def __init__(self, train_dl, valid_dl, c=None):
    self.train_dl, self.valid_dl, self.c = train_dl,valid_dl,c

  @property
  def train_ds(self): return self.train_dl.dataset

  @property
  def valid_ds(self): return self.valid_ds.dataset

In [23]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [29]:
#export
def get_model(data, lr=0.5, nh=50):
  m = data.train_ds.x.shape[1]
  model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
  return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
  def __init__(self, model, opt, loss_func, data):
    self.model, self.opt, self.loss_func, self.data = model,opt,loss_func,data

In [30]:
learn =  Learner(*get_model(data), loss_func, data)

In [36]:
def fit(epochs, learn):
  for epoch in range(epochs):
    learn.model.train()
    for xb,yb in learn.data.train_dl:
      loss = learn.loss_func(learn.model(xb), yb)
      loss.backward()
      learn.opt.step()
      learn.opt.zero_grad()

    learn.model.eval()
    with torch.no_grad():
      tot_loss, tot_acc  = 0.,0.
      for xb,yb in learn.data.valid_dl:
        pred = learn.model(xb)
        tot_loss += learn.loss_func(pred, yb)
        tot_acc += accuracy(pred,yb)

    nv = len(learn.data.valid_dl)
    print(epoch, tot_loss/nv, tot_acc/nv)
  return tot_loss/nv, tot_acc/nv


In [37]:
loss, acc = fit(10, learn)

0 tensor(0.0988) tensor(0.9708)
1 tensor(0.1312) tensor(0.9604)
2 tensor(0.3001) tensor(0.9241)
3 tensor(0.2979) tensor(0.9208)
4 tensor(0.1087) tensor(0.9686)
5 tensor(0.1080) tensor(0.9718)
6 tensor(0.1008) tensor(0.9718)
7 tensor(0.1041) tensor(0.9721)
8 tensor(1.2897) tensor(0.8184)
9 tensor(0.1055) tensor(0.9740)


**CALLBACK HANDLER**

In [38]:
def one_batch(xb, yb, cb):
  if not cb.begin_batch(xb,yb): return
  loss = cb.learn.loss_func(cb.learn.model(xb), yb)
  if not cb.after_loss(loss): return
  loss.backward()
  if cb.after_backward(): cb.learn.opt.step()
  if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
  for xb,yb in dl:
    one_batch(xb, yb, cb)
    if cb.do_stop(): return

def fit(epochs, learn, cb):
  if not cb.begin_fit(learn): return
  for epoch in range(epochs):
    if not cb.begin_epoch(epoch): continue
    all_batches(learn.data.train_dl, cb)

    if cb.begin_validate():
      with torch.no_grad(): all_batches(learn.data.valid_dl, cb)
    if cb.do_stop() or not cb.after_epoch(): break
  cb.after_fit()

In [39]:
class Callback():
  def begin_fit(self, learn):
    self.learn = learn
    return True
  def after_fit(self): return True
  def begin_epoch(self, epoch):
    self.epoch = epoch
  def begin_validate(self): return True
  def after_epoch(self): return True
  def begin_batch(self, xb, yb):
    self.xb, self.yb = xb, yb
    return True
  def after_loss(self, loss):
    self.loss = loss
    return True
  def after_backwards(self): return True
  def after_step(self): return True

In [40]:
class CallbackHandler():
  def __init__(self, cbs=None):
    self.cbs = cbs if cbs else []

  def begin_fit(self, learn):
    self.learn, self.in_train = learn, True
    learn.stop = False
    res = True
    for cb in self.cbs: res = res and cb.begin_fit(learn)

  def after_fit(self):
    res = not self.in_train
    for cb in self.cbs: res and cb.after_fit()
    return res

  def begin_epoch(self, epoch):
    learn.model.train()
    self.in_train=True
    res= True
    for cb in self.cbs: res = res and cb.begin_epoch(epoch)
    return res

  def begin_validate(self):
    learn.model.eval()
    self.in_train=False
    res= True
    for cb in self.cbs: res = res and cb.begin_validate(epoch)
    return res

  def after_epoch(self):
    res = True
    for cb in self.cbs: res = res and cb.after_epoch
    return res

  def begin_batch(self, xb, yb):
    res = True
    for cb in cbs: res = res and cb.begin_batch(xb,yb)
    return res

  def after_loss(self, loss):
    res = self.in_train
    for cb in self.cbs: res = res and cb.after_loss(loss)
    return res

  def after_backwards(self):
    res = True
    for cb in self.cbs: res = res and cb.after_backward()
    return res

  def after_step(self):
    res = True
    for cb in self.cbs: res = res and cb.after_step()
    return res

  def do_stop(self):
    try:      return learn
    finally:  learn.stop = False

In [41]:
class TestCallback(Callback):
  def begin_fit(self, learn):
    super().begin_fit(learn)
    self.n_iters = 0
    return True
  
  def after_step(self):
    self.n_iters += 1 
    print(self.n_iters)
    if self.n_iters >=10: learn.stop =True
    return True

In [44]:
fit(15, learn, cb = CallbackHandler([TestCallback()]))